# Fundamentos de los LLMs

*(Tomado y adaptado del [Deep Learning Notebook](https://colab.research.google.com/github/khipu-ai/practicals-2025/blob/main/notebooks/Foundations_of_LLMs.ipynb))*

<a href="https://colab.research.google.com/github/khipu-ai/practicals-2025/blob/main/notebooks/Foundations_of_LLMs.ipynb" target="_parent"><img src="https://colab.research.google.com/assets/colab-badge.svg" alt="Abrir en Colab"/></a>


## Introducción

En este tutorial, profundizarás en los principios fundamentales de los *transformers* y en la tecnología de vanguardia detrás de modelos como GPT.  
También tendrás experiencia práctica entrenando tu propio Modelo de Lenguaje (*Language Model*).  
Prepárate para explorar cómo estos impresionantes sistemas de IA generan texto tan realista y atractivo.  
¡Emprendamos juntos este emocionante viaje y descubramos los secretos de los LLMs! 🚀📚

A lo largo del tutorial hay una mezcla de ejemplos y ejercicios:
* Los ejercicios están marcados con 🛠️
* Los ejemplos y el código necesario para ejecutar el tutorial están marcados con ⚙️


## ¿Qué aprenderás?

### Temas

- <font color='green'> 🌱 Introducción: demostración usando Hugging Face</font>  
- <font color='orange'> 🪴 Comprensión del mecanismo de atención (*attention*)</font>  
- <font color='orange'> 🪴 Arquitectura Transformer</font>  
- <font color='orange'> 🪴 Objetivo de entrenamiento</font>  
- <font color='blue'> 🌳 Entrenamiento de tu propio LLM desde cero</font>  

- <font color='green'> 🌱 *Finetuning* de un LLM para Clasificación de Texto</font>

Nivel: <font color='green'>Principiante</font>, <font color='orange'>Intermedio</font>, <font color='blue'>Avanzado</font>

### Objetivos de Aprendizaje

* Comprender la idea detrás de [Attention](https://arxiv.org/abs/1706.03762) y por qué se utiliza.
* Presentar y describir los componentes fundamentales de la [Arquitectura Transformer](https://arxiv.org/abs/1706.03762) y la intuición detrás de su diseño.
* Construir y entrenar un simple LLM inspirado en Shakespeare.

## <font color='orange'> ⚠️ **Antes de comenzar:** </font>

Para esta práctica, necesitarás usar una **GPU** para acelerar el entrenamiento.  
Para activarla en Colab:
1. Ve al menú "Runtime".
2. Selecciona "Change runtime type".
3. En el menú emergente, elige **GPU** en la casilla "Hardware accelerator".


# Explorando Hugging Face

Antes de entrar en la parte práctica, hagamos una breve parada en el fascinante mundo de [Hugging Face](https://huggingface.co/): una plataforma abierta y colaborativa que ha revolucionado la forma en que trabajamos con modelos de lenguaje. 🌐

Para empezar a calentar motores, vamos a cargar un modelo de lenguaje **pequeño** (comparado con los gigantes actuales) y darle una instrucción sencilla.  
Esto te ayudará a entender cómo interactuar con estas bibliotecas tan potentes.  
Con solo unas pocas líneas de código, podrás descubrir todo el potencial que tienen los modelos de lenguaje. 💡

<img src="https://huggingface.co/datasets/huggingface/brand-assets/resolve/main/hf-logo.png" width="10%">

---

## ¿Qué es Hugging Face?

[Hugging Face](https://huggingface.co/) nació en 2016 con una misión muy clara:  
**"Hacer que el *machine learning* de calidad sea accesible para todos, un *commit* a la vez."**

Hoy en día es un recurso increíble para trabajar con modelos de lenguaje y no solo eso:  
- Han desarrollado varias bibliotecas de código abierto.  
- Permiten usar fácilmente modelos *transformers* preentrenados (de texto, imágenes, audio y más).  
- También ofrecen datasets listos para entrenar o ajustar (*fine-tuning*) estos modelos.  

Su software se usa muchísimo en la industria y en investigación.  
Si quieres profundizar más en su historia y en cómo se usan, puedes revisar [este práctico de 2022 sobre atención y transformers](https://colab.research.google.com/github/deep-learning-indaba/indaba-pracs-2022/blob/main/practicals/attention_and_transformers.ipynb#scrollTo=qFBw8kRx-4Mk).

---

### Entonces… ¿listo/a para empezar?  
Vamos a dar nuestros primeros pasos con Hugging Face y a jugar un poco con un modelo de lenguaje. ¡Manos a la obra!


In [ ]:
#@title ⚙️ Instalación e importación de librerías

# Instalamos las librerías necesarias para deep learning, NLP y visualización
!pip install transformers datasets      # Transformers y datasets: modelos y datos para tareas de NLP
!pip install seaborn umap-learn         # Seaborn para gráficas, UMAP para reducción de dimensionalidad
!pip install livelossplot               # LiveLossPlot para monitorear el progreso del entrenamiento
!pip install -q transformers[torch]     # Transformers con backend de PyTorch
!pip install -q peft                    # PEFT: Fine-tuning eficiente en parámetros
!pip install accelerate -U              # Accelerate: optimización de rendimiento en entrenamiento
!pip install gensim
# Herramientas adicionales para depuración y formato en consola
!pip install -q ipdb                    # Depurador interactivo de Python
!pip install -q colorama                # Texto con colores en la terminal

# --- Importación de librerías y configuraciones ---

# Utilidades del sistema y matemáticas básicas
import os
import math
import urllib.request

# Verificamos si hay aceleradores conectados (GPU o TPU)
if os.environ.get("COLAB_GPU") and int(os.environ["COLAB_GPU"]) > 0:
    print("Hay una GPU conectada.")
elif "COLAB_TPU_ADDR" in os.environ and os.environ["COLAB_TPU_ADDR"]:
    print("Hay una TPU conectada.")
    import jax.tools.colab_tpu
    jax.tools.colab_tpu.setup_tpu()
else:
    print("Solo hay CPU disponible.")

# Evitamos que JAX reserve toda la memoria de la GPU automáticamente
os.environ['XLA_PYTHON_CLIENT_PREALLOCATE'] = "false"

# Librerías para deep learning con JAX
import chex
import flax
import flax.linen as nn
import jax
import jax.numpy as jnp
from jax import grad, jit, vmap
import optax

# Librerías para NLP y modelos preentrenados
import transformers
from transformers import pipeline, AutoTokenizer, AutoModel
import datasets
import peft

# Procesamiento de imágenes y visualización
from PIL import Image
from livelossplot import PlotLosses
import matplotlib.pyplot as plt
import numpy as np
import seaborn as sns

# Utilidades adicionales para trabajar con texto y modelos
import torch
import torchvision
import itertools
import random
import copy

# Descargamos una imagen de ejemplo para usar en el notebook
urllib.request.urlretrieve(
    "https://images.unsplash.com/photo-1529778873920-4da4926a72c2?ixlib=rb-1.2.1&ixid=MnwxMjA3fDB8MHxzZWFyY2h8MXx8Y3V0ZSUyMGNhdHxlbnwwfHwwfHw%3D&w=1000&q=80",
    "cat.png",
)

# Preprocesamiento de texto y modelos preentrenados (Word2Vec, etc.)
import gensim
from nltk.data import find
import nltk
nltk.download("word2vec_sample")

# Herramientas de Hugging Face y widgets para interacción en notebooks
import huggingface_hub
import ipywidgets as widgets
from IPython.display import display
import colorama

# Configuramos Matplotlib para generar gráficas en formato SVG (mejor calidad)
%config InlineBackend.figure_format = 'svg'


Hay una GPU conectada.


[nltk_data] Downloading package word2vec_sample to /root/nltk_data...
[nltk_data]   Unzipping models/word2vec_sample.zip.


In [ ]:
#@title ⚙️ Funciones auxiliares para graficar

def plot_position_encodings(P, max_tokens, d_model):
    """
    Grafica la matriz de codificaciones posicionales.

    Parámetros:
        P: Matriz de codificaciones posicionales (arreglo 2D).
        max_tokens: Número máximo de tokens (filas) a graficar.
        d_model: Dimensionalidad del modelo (columnas) a graficar.
    """

    # Ajustamos el tamaño de la figura según el número de tokens y dimensiones
    plt.figure(figsize=(20, np.min([8, max_tokens])))

    # Graficamos la matriz de codificaciones posicionales con un mapa de colores
    im = plt.imshow(P, aspect="auto", cmap="Blues_r")

    # Agregamos una barra de color para indicar los valores de las codificaciones
    plt.colorbar(im, cmap="blue")

    # Mostramos los índices de las dimensiones (ejes x) si la dimensionalidad es pequeña
    if d_model <= 64:
        plt.xticks(range(d_model))

    # Mostramos los índices de las posiciones (eje y) si el número de tokens es pequeño
    if max_tokens <= 32:
        plt.yticks(range(max_tokens))

    # Etiquetamos los ejes
    plt.xlabel("Índice de la incrustación (embedding)")
    plt.ylabel("Índice de la posición")

    # Mostramos la gráfica
    plt.show()


def plot_image_patches(patches):
    """
    Recibe una lista o arreglo de parches de imagen y los grafica.

    Parámetros:
        patches: Lista o arreglo de parches de imagen a graficar.
    """

    # Configuramos la figura para graficar los parches
    fig = plt.figure(figsize=(25, 25))

    # Creamos un subplot para cada parche y lo mostramos
    axes = []
    for a in range(patches.shape[1]):
        axes.append(fig.add_subplot(1, patches.shape[1], a + 1))
        plt.imshow(patches[0][a])

    # Ajustamos el diseño para evitar superposición y mostramos la gráfica
    fig.tight_layout()
    plt.show()


def plot_projected_embeddings(embeddings, labels):
    """
    Proyecta embeddings de alta dimensión a un espacio 2D y los grafica.

    Parámetros:
        embeddings: Vectores de alta dimensión a proyectar.
        labels: Etiquetas correspondientes a cada embedding para colorear los puntos.
    """

    # Importamos UMAP y Seaborn para reducción de dimensionalidad y graficado
    import umap
    import seaborn as sns

    # Reducimos la dimensionalidad a 2D usando UMAP
    projected_embeddings = umap.UMAP().fit_transform(embeddings)

    # Graficamos las proyecciones 2D con Seaborn para mejor estética
    plt.figure(figsize=(15, 8))
    plt.title("Proyección de los embeddings de texto")
    sns.scatterplot(
        x=projected_embeddings[:, 0], y=projected_embeddings[:, 1], hue=labels
    )

    # Mostramos la gráfica
    plt.show()


def plot_attention_weight_matrix(weight_matrix, x_ticks, y_ticks):
    """
    Grafica una matriz de pesos de atención con etiquetas personalizadas en los ejes.

    Parámetros:
        weight_matrix: Matriz de pesos de atención a graficar.
        x_ticks: Etiquetas para el eje x (normalmente tokens de consulta o query).
        y_ticks: Etiquetas para el eje y (normalmente tokens clave o key).
    """

    # Ajustamos el tamaño de la figura
    plt.figure(figsize=(15, 7))

    # Graficamos la matriz de pesos de atención como un mapa de calor
    ax = sns.heatmap(weight_matrix, cmap="Blues")

    # Colocamos etiquetas personalizadas en los ejes
    plt.xticks(np.arange(weight_matrix.shape[1]) + 0.5, x_ticks)
    plt.yticks(np.arange(weight_matrix.shape[0]) + 0.5, y_ticks)

    # Etiquetamos la gráfica
    plt.title("Matriz de atención")
    plt.xlabel("Puntaje de atención")

    # Mostramos la gráfica
    plt.show()


In [ ]:
#@title ⚙️ Funciones auxiliares para procesamiento de texto

def get_word2vec_embedding(words):
    """
    Obtiene los embeddings de una lista de palabras usando un modelo
    preentrenado de word2vec.

    Parámetros:
        words: Lista de palabras a convertir en embeddings.

    Retorna:
        embeddings: Arreglo con los vectores de embeddings.
        words_pass: Lista de palabras que fueron procesadas correctamente.
    """

    # Cargamos un modelo word2vec de ejemplo incluido en NLTK
    word2vec_sample = str(find("models/word2vec_sample/pruned.word2vec.txt"))
    model = gensim.models.KeyedVectors.load_word2vec_format(
        word2vec_sample, binary=False
    )

    output = []
    words_pass = []
    for word in words:
        try:
            # Si la palabra existe en el modelo, obtenemos su embedding
            output.append(jnp.array(model.word_vec(word)))
            words_pass.append(word)
        except:
            # Si la palabra no está en el vocabulario, la ignoramos
            pass

    embeddings = jnp.array(output)
    del model  # Liberamos memoria
    return embeddings, words_pass


def remove_punctuation(text):
    """
    Elimina todos los signos de puntuación de un texto.

    Parámetros:
        text: Cadena de texto de entrada.

    Retorna:
        Texto limpio sin puntuación.
    """
    import re
    text = re.sub(r"[^\w\s]", "", text)
    return text


def print_sample(prompt: str, sample: str):
    """
    Imprime un ejemplo con el prompt y la respuesta del modelo,
    usando colores diferentes para distinguirlos visualmente.

    Parámetros:
        prompt: Instrucción o entrada dada al modelo.
        sample: Respuesta generada por el modelo.
    """
    print(colorama.Fore.MAGENTA + prompt, end="")
    print(colorama.Fore.BLUE + sample)
    print(colorama.Fore.RESET)


### Demostración de LLM: Cargar e interactuar con un modelo de Hugging Face

Veamos lo sencillo que es cargar e interactuar con un modelo desde Hugging Face.

Para este tutorial, hemos preconfigurado dos opciones de modelos:

- **`gpt-neo-125M`**: Un modelo pequeño con 125 millones de parámetros. Es rápido y consume poca memoria, ideal para empezar. Recomendamos probar este primero.
- **`gpt2-medium`**: Un modelo más grande con 355 millones de parámetros, útil si quieres respuestas un poco más sofisticadas.

Si deseas cambiar de modelo, basta con reiniciar el kernel de Colab y actualizar el nombre del modelo en la celda correspondiente.

**Nota**: Estos pasos no solo funcionan con estos dos modelos, sino también con [todos los modelos](https://huggingface.co/models?pipeline_tag=text-generation) de Hugging Face que soporten el pipeline de generación de texto.


In [ ]:
#@markdown En este Colab, imprimimos los *prompts* en <font color='HotPink'><b>rosa</b></font> y las respuestas generadas por el modelo en <font color='blue'><b>azul</b></font>, como en el ejemplo siguiente:

print_sample(prompt='Mi prompt falso', sample=' es genial!')


Mi prompt falso es genial!



In [ ]:
#@title ⚙️ Demo
#@markdown Datasets Package - <font color='blue'>`Beginner`</font>

# Set the model name to "EleutherAI/gpt-neo-125M" (this can be changed via the dropdown options)
MODEL_NAME = "gpt2-medium"  # @param ["gpt2-medium", "EleutherAI/gpt-neo-125M"]

# Define the prompt for the text generation model
test_prompt = 'What is love?'  # @param {type: "string"}

# Create a text generation pipeline using the specified model
generator = transformers.pipeline('text-generation',
                                  model=MODEL_NAME)

# Generate text based on the provided prompt
# 'do_sample=True' enables sampling to introduce randomness in generation, and 'min_length=30' ensures at least 30 tokens are generated
model_output = generator(test_prompt, do_sample=True, min_length=30)

# Print the generated text sample, removing the original prompt from the output
print_sample(test_prompt, model_output[0]['generated_text'].split(test_prompt)[1].rstrip())

/usr/local/lib/python3.11/dist-packages/huggingface_hub/utils/_auth.py:94: UserWarning: 
The secret `HF_TOKEN` does not exist in your Colab secrets.
To authenticate with the Hugging Face Hub, create a token in your settings tab (https://huggingface.co/settings/tokens), set it as secret in your Google Colab and restart your session.
You will be able to reuse this secret in all of your notebooks.
Please note that authentication is recommended but still optional to access public models or datasets.
  warnings.warn(


config.json:   0%|          | 0.00/718 [00:00<?, ?B/s]

model.safetensors:   0%|          | 0.00/1.52G [00:00<?, ?B/s]

generation_config.json:   0%|          | 0.00/124 [00:00<?, ?B/s]

tokenizer_config.json:   0%|          | 0.00/26.0 [00:00<?, ?B/s]

vocab.json:   0%|          | 0.00/1.04M [00:00<?, ?B/s]

merges.txt:   0%|          | 0.00/456k [00:00<?, ?B/s]

tokenizer.json:   0%|          | 0.00/1.36M [00:00<?, ?B/s]

Device set to use cuda:0
Setting `pad_token_id` to `eos_token_id`:50256 for open-end generation.


What is love? What is happiness?"

"Love is the love of others and the love of the soul."

"Love is the freedom of the soul and the freedom of heart."

"Love is the love of the heart and the freedom of the body."

"Love is the love of the soul and the freedom of the body."

The Church has always taught that the essential aim of life is to attain to the unity of life, and that the life of the soul is a constant union with the life of the body in order to achieve the unity of life. The Church has always called love the supreme, the supreme goal of all human life.

"The Church believes that the greatest of all human aims is to attain to a union of the soul with the body, the union of the body with the soul, the union of the soul with God."

"The Church will never allow anything to stand in the way of that union."

"Whoever, however, desires to make a union of the body and the soul, it is not enough for the soul to love the body, but she must also love God."

"The love of God does not cons

**Consejo:** Prueba ejecutar el código anterior con diferentes *prompts* o incluso con el mismo *prompt* varias veces.

**Para reflexionar:** ¿Por qué crees que el texto generado cambia cada vez, incluso cuando usas el mismo *prompt*?  
Escribe tu respuesta en el campo de entrada de abajo y comparte tu opinión con tu compañero/a.


In [ ]:
#@title ⚙️ Personaliza tu pipeline de generación de texto

def get_model_and_tokenizer(model_name: str):
    """
    Dado el nombre de un modelo, carga el modelo y el tokenizador correspondientes.

    Parámetros:
        model_name: Nombre del modelo a cargar.

    Retorna:
        model: El modelo de lenguaje que usaremos para generar texto.
        tokenizer: El tokenizador que convierte texto en el formato que el modelo entiende.
    """
    match model_name:
        case 'gpt2-medium':
            # Cargamos el tokenizador y modelo GPT-2
            tokenizer = transformers.GPT2Tokenizer.from_pretrained(model_name)
            model = transformers.GPT2LMHeadModel.from_pretrained(model_name)
        case 'EleutherAI/gpt-neo-125M':
            tokenizer = transformers.AutoTokenizer.from_pretrained(model_name)
            model = transformers.AutoModelForCausalLM.from_pretrained(model_name)
        case _:
            raise NotImplementedError(f'El modelo {model_name} no está reconocido')

    # Si hay GPU disponible, movemos el modelo a la GPU para acelerar el procesamiento
    if torch.cuda.is_available():
        model = model.to("cuda")

    # Ajustamos el token de padding para que sea igual al token de fin de secuencia
    tokenizer.pad_token_id = tokenizer.eos_token_id

    return model, tokenizer


def run_sample(
    model_name: str,
    prompt: str,
    seed: int | None = None,
    temperature: float = 0.6,
    top_p: float = 0.9,
    max_new_tokens: int = 64,
) -> str:
    """
    Ejecuta el modelo para generar texto a partir de un prompt.

    Esta función genera texto usando un modelo de lenguaje, con opciones para controlar
    la aleatoriedad, la cantidad máxima de tokens generados y la reproducibilidad.

    Parámetros:
        model_name: Nombre del modelo a usar.
        prompt: Texto de entrada para que el modelo comience la generación.
        seed: Número para fijar la semilla y hacer los resultados reproducibles (opcional).
        temperature: Controla la aleatoriedad en la generación; valores bajos dan texto más predecible.
        top_p: Controla la diversidad considerando solo las palabras más probables hasta cubrir este porcentaje.
        max_new_tokens: Máximo número de tokens a generar luego del prompt.

    Retorna:
        Texto generado por el modelo.
    """
    model, tokenizer = get_model_and_tokenizer(model_name)

    # Convertimos el texto del prompt a tokens que el modelo pueda procesar
    inputs = tokenizer(prompt, return_tensors="pt")

    # Extraemos los IDs de los tokens y la máscara de atención
    input_ids = inputs["input_ids"]
    attention_mask = inputs["attention_mask"]

    # Movemos los tensores al dispositivo donde está el modelo (GPU o CPU)
    input_ids = input_ids.to(model.device)
    attention_mask = attention_mask.to(model.device)

    # Configuramos los parámetros para la generación de texto
    generation_config = transformers.GenerationConfig(
        do_sample=True,          # Permite que el modelo genere con aleatoriedad
        temperature=temperature, # Controla cuán aleatorio es el resultado (más bajo = menos aleatorio)
        top_p=top_p,             # Considera solo el conjunto de palabras que cubren top_p de probabilidad
        pad_token_id=tokenizer.pad_token_id, # Token para padding
        top_k=0,                 # No limitamos la cantidad top_k de palabras
    )

    # Si se fija una semilla, la configuramos para que la generación sea reproducible
    if seed is not None:
        torch.manual_seed(seed)

    # Generamos texto con el modelo y las configuraciones definidas
    generation_output = model.generate(
        input_ids=input_ids,
        attention_mask=attention_mask,
        return_dict_in_generate=True,
        output_scores=True,
        max_new_tokens=max_new_tokens,
        generation_config=generation_config,
    )

    # Nos aseguramos que solo se haya generado una secuencia para simplificar
    assert len(generation_output.sequences) == 1

    # Obtenemos la secuencia generada
    output_sequence = generation_output.sequences[0]

    # Decodificamos los tokens generados a texto legible
    output_string = tokenizer.decode(output_sequence)

    # Imprimimos el prompt y la respuesta generada con colores para distinguirlos
    print_sample(prompt, output_string)

    # Retornamos el texto generado
    return output_string


In [ ]:
_ = run_sample(model_name=MODEL_NAME, prompt="What is love?", temperature = 0.5, seed=2)

What is love?What is love? It is the love that is given to another, the love that is given to you, the love that is given to me. I love you.

Love is the love that is given to another, the love that is given to you, the love that is given to me.

Love is the love



Es bastante impresionante, cierto? Prueba a jugar con los valores de **prompt**, **temperature** y **seed** en el código anterior y observa las diferentes respuestas que obtienes. ¿Qué notas cuando aumentas la temperatura?

Aunque esto pudo parecer revolucionario en 2021, probablemente muchos de ustedes ya han interactuado con modelos de lenguaje grandes de alguna forma. Hoy vamos a dar un paso más allá y entrenaremos nuestro propio **modelo de lenguaje inspirado en Shakespeare**. Esto nos permitirá entender de manera práctica cómo funcionan estos modelos por dentro.

Pero antes de comenzar con el entrenamiento, construyamos una base sólida sobre qué son los **Modelos de Lenguaje Grandes (LLMs)** y los conceptos clave de **Aprendizaje Automático** que hacen posible esta tecnología innovadora. En el núcleo de los modelos LLM de última generación están el **Mecanismo de Atención** y la **Arquitectura Transformer**. Exploraremos estos conceptos esenciales en las siguientes secciones de este tutorial.
